## Primary & Blooming Model

In [1]:
from keras.models import Sequential, load_model, Model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16, preprocess_input

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
from sklearn.externals import joblib

import numpy as np
import cPickle as pickle
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
sys.path.insert(0, '..')
import paths
sys.path.insert(0, '../rainforest')
import data
import data_generators_leo as dgl

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 1050 Ti (0000:09:00.0)


### Load training data and labels.

In [10]:
train_csv = data.get_class_data(train=True, label='primary')
val_csv = data.get_class_data(train=False, label='primary')
train_labels = np.asarray(train_csv)[:,1:3] 
train_data = np.asarray(train_csv)[:,0] #name of image
all_zeros_train = [index for index, label in enumerate(train_labels) if sum(label) == 0]
if(all_zeros_train > 0):
    train_data = np.delete(train_data, all_zeros_train)
    train_labels = np.delete(train_labels, all_zeros_train, axis=0)

val_labels = np.asarray(val_csv)[:,1:5]
val_data = np.asarray(val_csv)[:,0]
all_zeros_val = [index for index, label in enumerate(val_labels) if sum(label) == 0]
if(all_zeros_val > 0):
    val_data = np.delete(val_data, all_zeros_val)
    val_labels = np.delete(val_labels, all_zeros_val, axis=0)

labels=['primary', 'blooming']

#data is [image name]
train_labels = train_labels[:-1]
train_data = train_data[:-1]
val_labels = val_labels[:-1]
val_data = val_data[:-1]
print(train_labels.shape)
print(train_data.shape)
print(val_labels.shape)
print(val_data.shape)

(26283, 2)
(26283,)
(11228, 2)
(11228,)


### Generators

In [11]:
train_gen = dgl.get_data(train_data, '../data/train-jpg', train_labels,
                                      flip=False, shift_x=0, shift_y=0, rot_range=0,
                                      elastic_trans=False, batch_size=10, balance_batches=True)
val_gen = dgl.get_data(val_data, '../data/train-jpg', val_labels, batch_size=8)

### Network Structure

In [12]:
model = Sequential()

model.add(BatchNormalization(input_shape=[3, 256, 256], axis=1))

model.add(Convolution2D(32, (3, 3), input_shape=[3, 256, 256], activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Convolution2D(64, 3, 3, activation='relu'))
#model.add(BatchNormalization(axis=1))
#model.add(Convolution2D(64, 3, 3, activation='relu'))
#model.add(BatchNormalization(axis=1))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Convolution2D(128, 3, 3, activation='relu'))
#model.add(BatchNormalization(axis=1))
#model.add(Convolution2D(128, 3, 3, activation='relu'))
#model.add(BatchNormalization(axis=1))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
#model.add(Dense(128, activation='relu'))
model.add(Dense(len(labels), activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy', 'categorical_crossentropy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_7 (Batch (None, 3, 256, 256)       12        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 254, 254)      896       
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 254, 254)      128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 252, 252)      9248      
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 252, 252)      128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 126, 126)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 508032)            0         
__________

### Training Network

In [13]:
model.fit_generator(train_gen, steps_per_epoch=len(train_data)/100,
                    epochs=5, verbose=1,
                    validation_data=val_gen, validation_steps=len(val_data)/100)

Epoch 1/5


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/pieter/anaconda2/envs/mlippython2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/pieter/anaconda2/envs/mlippython2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/pieter/anaconda2/envs/mlippython2/lib/python2.7/site-packages/keras/engine/training.py", line 606, in data_generator_task
    generator_output = next(self._generator)
  File "../rainforest/data_generators_leo.py", line 50, in get_data
    label = labels[i]
IndexError: index 26283 is out of bounds for axis 0 with size 26283



ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None